In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
validation_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\Interagency Tracking System.gdb"
output_gdb_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\tmp\CNRA_2020_2025.gdb"

In [3]:
'CNRA_enriched_ln_20241121'
'CNRA_enriched_poly_20241121'
'CNRA_enriched_pt_20241121'

'CNRA_enriched_pt_20241121'

In [4]:
'CNRA_enriched_20250205_line'
'CNRA_enriched_20250205_polygon'
'CNRA_enriched_20250205_point'

'CNRA_enriched_20250205_point'

In [5]:
reference_gdf = gpd.read_file(validation_gdb_path, driver='openFileGDB', layer='CNRA_enriched_poly_20241121')
output_gdf = gpd.read_file(output_gdb_path, driver='openFileGDB', layer='CNRA_enriched_20250205_polygon')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts.  The processing may be really slow.  You can skip the processing by setting METHOD=SKIP.
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [6]:
reference_gdf.columns

Index(['globalid', 'projectid', 'trmtid_user', 'projectid_user',
       'projectname_', 'org_admin_t', 'primary_ownership_group',
       'primary_objective', 'secondary_objective', 'tertiary_objective',
       'treatment_status', 'county', 'in_wui', 'region', 'treatment_area',
       'treatment_start', 'treatment_end', 'retreatment_date_est',
       'treatment_name', 'ACTIVID_USER', 'TREATMENTID_', 'ORG_ADMIN_a',
       'ACTIVITY_DESCRIPTION', 'ACTIVITY_CAT', 'BROAD_VEGETATION_TYPE',
       'BVT_USERD', 'ACTIVITY_STATUS', 'ACTIVITY_QUANTITY', 'ACTIVITY_UOM',
       'ACTIVITY_START', 'ACTIVITY_END', 'ADMIN_ORG_NAME', 'IMPLEM_ORG_NAME',
       'PRIMARY_FUND_SRC_NAME', 'PRIMARY_FUND_ORG_NAME',
       'SECONDARY_FUND_SRC_NAME', 'SECONDARY_FUND_ORG_NAME',
       'TERTIARY_FUND_SRC_NAME', 'TERTIARY_FUND_ORG_NAME', 'ACTIVITY_PRCT',
       'RESIDUE_FATE', 'RESIDUE_FATE_QUANTITY', 'RESIDUE_FATE_UNITS',
       'ACTIVITY_NAME', 'VAL_STATUS_a', 'VAL_MSG_a', 'VAL_RUNDATE_a',
       'REVIEW_STATUS_a

In [7]:
out_group = output_gdf[output_gdf['COUNTS_TO_MAS'] == 'YES'][['ACTIVITY_QUANTITY', 'ADMINISTERING_ORG','PRIMARY_OBJECTIVE', "Year_txt"]].groupby(['ADMINISTERING_ORG','PRIMARY_OBJECTIVE', "Year_txt"]).sum().reset_index()

In [8]:
ref_group = reference_gdf[reference_gdf['COUNTS_TO_MAS'] == 'YES'][['ACTIVITY_QUANTITY', 'administering_org','primary_objective', "Year_txt"]].groupby(['administering_org','primary_objective', "Year_txt"]).sum().reset_index()

In [9]:
ref_group = ref_group.rename(columns={'administering_org': 'administering_org'.upper(), 'primary_objective': 'primary_objective'.upper()})

In [10]:
merge_group = out_group.merge(ref_group, on=['ADMINISTERING_ORG','PRIMARY_OBJECTIVE', "Year_txt"], how='outer')

In [11]:
sum(merge_group['ACTIVITY_QUANTITY_x'].fillna(0) - merge_group['ACTIVITY_QUANTITY_y'].fillna(0))

15315.812742499998

In [12]:
pd.set_option('display.max_rows', None)
merge_group[merge_group['ACTIVITY_QUANTITY_x'] != merge_group['ACTIVITY_QUANTITY_y']]

,ADMINISTERING_ORG,PRIMARY_OBJECTIVE,Year_txt,ACTIVITY_QUANTITY_x,ACTIVITY_QUANTITY_y
54,CCC,OTHER_FUELS_REDUCTION,2025,25.790000,NaN
55,CCC,OTHER_FUELS_REDUCTION,2027,NaN,25.790000
57,CCC,TBD,2023,NaN,2.250000
58,CDFW,BURNED_AREA_RESTOR,2023,11.000000,33.000000
59,CDFW,BURNED_AREA_RESTOR,2024,8.000000,32.000000
61,CDFW,FUEL_BREAK,2022,8638.989002,8640.489002
62,CDFW,FUEL_BREAK,2023,9228.834158,9235.034158
63,CDFW,FUEL_BREAK,2024,8515.015635,8523.015635
68,CDFW,INV_SPECIES_CNTRL,2021,NaN,0.100000
69,CDFW,INV_SPECIES_CNTRL,2022,6377.900000,6378.120000
